In [1]:
from dichasus_cf0x import training_set, spec
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import tqdm

2024-02-10 16:38:43.702320: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-10 16:38:44.478801: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-02-10 16:38:45.458548: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-10 16:38:45.482729: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
generator = tf.keras.models.load_model('results/generator.keras')

In [3]:
NOISE_SHAPE = generator.layers[0].input_shape[0][1]
POS_SHAPE = generator.layers[1].input_shape[0][1]

### Use normalized test set coordinates as position coordinates

In [4]:
test_set_positions = np.load("results/test_set_positions.npy")
training_set_positions = np.load("results/training_set_positions.npy")
pos_xmin = np.min(training_set_positions[:,0])
pos_xmax = np.max(training_set_positions[:,0])
pos_ymin = np.min(training_set_positions[:,1])
pos_ymax = np.max(training_set_positions[:,1])

def normalize_pos(pos):
    x_norm = 2 * (pos[...,0] - pos_xmin) / (pos_xmax - pos_xmin) - 1
    y_norm = 2 * (pos[...,1] - pos_ymin) / (pos_ymax - pos_ymin) - 1
    return tf.stack([x_norm, y_norm], axis = -1)

pos_gpu = tf.constant(tf.cast(normalize_pos(test_set_positions), tf.float32), dtype = tf.float32)

### Fixed noise CSI

In [5]:
fixed_noise = tf.random.normal([NOISE_SHAPE])
fixed_noise_repeated = tf.tile(fixed_noise[tf.newaxis,:], [tf.shape(pos_gpu)[0], 1])
test_set_csi_gan_fixednoise = generator([fixed_noise_repeated, pos_gpu], training = False).numpy()
np.save("results/test_set_csi_gan_fixednoise.npy", test_set_csi_gan_fixednoise)

2024-02-10 16:38:48.582714: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


### Random noise CSI

In [6]:
random_noise = tf.random.normal([np.shape(pos_gpu)[0], NOISE_SHAPE])
test_set_csi_gan_randomnoise = generator([random_noise, pos_gpu], training = False).numpy()
np.save("results/test_set_csi_gan_randomnoise.npy", test_set_csi_gan_randomnoise)